In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 whic

In [3]:
import torch
from torch import nn
from transformers import CLIPModel, CLIPProcessor, GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

In [5]:
class ImageToHTMLModel(nn.Module):
    def __init__(self, clip_model_name="openai/clip-vit-base-patch32", gpt_model_name="gpt2"):
        super().__init__()
        self.clip = CLIPModel.from_pretrained(clip_model_name).vision_model
        self.gpt = GPT2LMHeadModel.from_pretrained(gpt_model_name)
        self.linear = nn.Linear(self.clip.config.hidden_size, self.gpt.config.n_embd)

    def forward(self, pixel_values, labels=None):
        image_features = self.clip(pixel_values).last_hidden_state[:, 0, :]
        image_features = self.linear(image_features)

        if labels is not None:
            outputs = self.gpt(inputs_embeds=image_features.unsqueeze(1), labels=labels)
        else:
            outputs = self.gpt.generate(inputs_embeds=image_features.unsqueeze(1), max_length=512)

        return outputs

def preprocess_data(example, processor, tokenizer):
    image = processor(images=example['image'], return_tensors="pt").pixel_values
    html = tokenizer(example['html'], truncation=True, max_length=512, padding="max_length", return_tensors="pt")
    return {"pixel_values": image.squeeze(), "labels": html.input_ids.squeeze()}

def train(model, train_dataloader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in tqdm(train_dataloader):
        pixel_values = batch['pixel_values'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(pixel_values, labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_dataloader)

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load dataset
    dataset = load_dataset("HuggingFaceM4/WebSight", split="train[:10]")  # Using a subset for demonstration

    # Initialize models and processors
    clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

    # Preprocess dataset
    processed_dataset = dataset.map(
        lambda example: preprocess_data(example, clip_processor, gpt_tokenizer),
        remove_columns=dataset.column_names
    )

    # Create data loader
    train_dataloader = DataLoader(processed_dataset, batch_size=8, shuffle=True)

    # Initialize model
    model = ImageToHTMLModel().to(device)

    # Training loop
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    num_epochs = 5

    for epoch in range(num_epochs):
        avg_loss = train(model, train_dataloader, optimizer, device)
        print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

    # Save the model
    torch.save(model.state_dict(), "image_to_html_model.pth")

if __name__ == "__main__":
    main()



Resolving data files:   0%|          | 0/738 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/738 [00:00<?, ?it/s]

(…)-00005-of-00738-89cf78e53b934db0.parquet:   0%|          | 0.00/408M [00:00<?, ?B/s]

(…)-00006-of-00738-ba36f1dbd3143674.parquet:   0%|          | 0.00/414M [00:00<?, ?B/s]

(…)-00007-of-00738-00b0a9a4836cf7a5.parquet:   0%|          | 0.00/411M [00:00<?, ?B/s]

KeyboardInterrupt: 